In [1]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from preprocess import pos_tagger

df = pd.read_csv('imdb.tsv', delimiter = "\\t")

df['sent_tokens'] = df['review'].apply(sent_tokenize)
df['pos_tagged_tokens'] = df['sent_tokens'].apply(pos_tagger)

df['pos_tagged_tokens']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\Song\AppData\Local\Temp\ipykernel_14648\1755282833.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('imdb.tsv', delimiter = "\\t")


0    [(``, ``), (Watching, JJ), (Time, NN), (Chaser...
1    [(I, PRP), (saw, VBD), (this, DT), (film, NN),...
2    [(Minor, JJ), (Spoilers, NNS), (In, IN), (New,...
3    [(I, PRP), (went, VBD), (to, TO), (see, VB), (...
4    [(``, ``), (Yes, UH), (,, ,), (I, PRP), (agree...
5    [(``, ``), (Jennifer, NNP), (Ehle, NNP), (was,...
6    [(Amy, NNP), (Poehler, NNP), (is, VBZ), (a, DT...
7    [(``, ``), (A, DT), (plane, NN), (carrying, VB...
8    [(A, DT), (well, NN), (made, VBN), (,, ,), (gr...
9    [(``, ``), (Incredibly, RB), (dumb, JJ), (and,...
Name: pos_tagged_tokens, dtype: object

In [4]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from preprocess import pos_tagger
from preprocess import penn_to_wn
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')

df = pd.read_csv('imdb.tsv', delimiter = "\t")
df['sent_tokens'] = df['review'].apply(sent_tokenize)
df['pos_tagged_tokens'] = df['sent_tokens'].apply(pos_tagger)

def swn_polarity(pos_tagged_sents):
    # 감성 지수 초기화
    sentiment_score = 0

    for word, tag in pos_tagged_sents:
        # NLTK 기반 품사 태깅을 WordNet기반 품사 태그로 변환
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue
            
        # 단어와 품사 태그를 기반으로 Synsets 구하기
        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            continue
        
        # Synsets의 첫 번째 요소의 이름으로 단일 SentiSynset 구하기
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        
        # SentiSynset을 통해 단어의 감성 지수 구하기
        word_senti_score = swn_synset.pos_score() - swn_synset.neg_score()

        # 각 단어의 감성 지수를 더하여 코퍼스의 감성 지수 값 계산해 반환하기
        sentiment_score += word_senti_score
    return sentiment_score

# 데이터 프레임에 적용 
df['swn_sentiment'] = df['pos_tagged_tokens'].apply(swn_polarity)

# 테스트 코드
df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Unnamed: 0,review,sent_tokens,pos_tagged_tokens,swn_sentiment
0,0,"Watching Time Chasers, it obvious that it was ...","[Watching Time Chasers, it obvious that it was...","[(Watching, VBG), (Time, NNP), (Chasers, NNPS)...",-0.250
1,1,I saw this film about 20 years ago and remembe...,[I saw this film about 20 years ago and rememb...,"[(I, PRP), (saw, VBD), (this, DT), (film, NN),...",-1.500
2,2,"Minor Spoilers In New York, Joan Barnard (Elvi...","[Minor Spoilers In New York, Joan Barnard (Elv...","[(Minor, JJ), (Spoilers, NNS), (In, IN), (New,...",-3.000
3,3,I went to see this film with a great deal of e...,[I went to see this film with a great deal of ...,"[(I, PRP), (went, VBD), (to, TO), (see, VB), (...",-0.250
4,4,"Yes, I agree with everyone on this site this m...","[Yes, I agree with everyone on this site this ...","[(Yes, UH), (,, ,), (I, PRP), (agree, VBP), (w...",2.625
5,5,"Jennifer Ehle was sparkling in \""Pride and Pre...","[Jennifer Ehle was sparkling in \""Pride and Pr...","[(Jennifer, NNP), (Ehle, NNP), (was, VBD), (sp...",7.000
6,6,Amy Poehler is a terrific comedian on Saturday...,[Amy Poehler is a terrific comedian on Saturda...,"[(Amy, NNP), (Poehler, NNP), (is, VBZ), (a, DT...",0.750
7,7,A plane carrying employees of a large biotech ...,[A plane carrying employees of a large biotech...,"[(A, DT), (plane, NN), (carrying, VBG), (emplo...",9.000
8,8,"A well made, gritty science fiction movie, it ...","[A well made, gritty science fiction movie, it...","[(A, DT), (well, NN), (made, VBN), (,, ,), (gr...",4.500
9,9,Incredibly dumb and utterly predictable story ...,[Incredibly dumb and utterly predictable story...,"[(Incredibly, RB), (dumb, JJ), (and, CC), (utt...",-1.625
